# Parsing ISO 8583 Messages

This notebook provides a deep dive into parsing ISO 8583 messages with iso8583sim.

## Setup

In [1]:
import sys

sys.path.insert(0, "..")

from iso8583sim.core.builder import ISO8583Builder
from iso8583sim.core.parser import ISO8583Parser
from iso8583sim.core.types import CardNetwork, ISO8583Message, get_field_definition

## Understanding Bitmaps

The bitmap is the key to parsing ISO 8583 messages. It tells us which fields are present.

### Primary Bitmap (Fields 1-64)

The primary bitmap is always present and covers fields 1-64. It's represented as 16 hex characters (64 bits).

In [2]:
def explain_bitmap(bitmap_hex: str) -> None:
    """Explain which fields are present in a bitmap."""
    bitmap_int = int(bitmap_hex, 16)
    present_fields = []

    for i in range(64):
        # Bit 0 (MSB) = field 1, Bit 63 (LSB) = field 64
        if bitmap_int & (1 << (63 - i)):
            present_fields.append(i + 1)

    print(f"Bitmap: {bitmap_hex}")
    print(f"Binary: {bin(bitmap_int)[2:].zfill(64)}")
    print(f"Present fields: {present_fields}")
    return present_fields


# Example: Bitmap with fields 2, 3, 4, 11, 41, 42
example_bitmap = "7020000000C00000"
explain_bitmap(example_bitmap)

Bitmap: 7020000000C00000
Binary: 0111000000100000000000000000000000000000110000000000000000000000
Present fields: [2, 3, 4, 11, 41, 42]


[2, 3, 4, 11, 41, 42]

### Secondary Bitmap (Fields 65-128)

If bit 1 of the primary bitmap is set, a secondary bitmap follows, covering fields 65-128.

In [3]:
# Example with secondary bitmap (bit 1 set = field 1 present)
primary_with_secondary = "F020000000C00000"  # Bit 1 set
fields = explain_bitmap(primary_with_secondary)
print("\nField 1 present means secondary bitmap follows!")

Bitmap: F020000000C00000
Binary: 1111000000100000000000000000000000000000110000000000000000000000
Present fields: [1, 2, 3, 4, 11, 41, 42]

Field 1 present means secondary bitmap follows!


## Field Types

ISO 8583 defines several field types:

| Type | Description | Example |
|------|-------------|--------|
| N | Numeric | `"123456"` |
| AN | Alphanumeric | `"ABC123"` |
| ANS | Alphanumeric + Special | `"Hello, World!"` |
| B | Binary | Hex-encoded bytes |
| Z | Track 2 data | `"4111111111111111=2512"` |

In [4]:
# Look up field definitions

interesting_fields = [2, 3, 4, 35, 39, 55]
print("Field Definitions:")
print("-" * 70)
for field_num in interesting_fields:
    field_def = get_field_definition(field_num)
    print(f"Field {field_num:3d}: {field_def.description}")
    print(f"          Type: {field_def.field_type.name}, Max Length: {field_def.max_length}")
    print()

Field Definitions:
----------------------------------------------------------------------
Field   2: Primary Account Number (PAN)
          Type: LLVAR, Max Length: 19

Field   3: Processing Code
          Type: NUMERIC, Max Length: 6

Field   4: Amount, Transaction
          Type: NUMERIC, Max Length: 12

Field  35: Track 2 Data
          Type: LLVAR, Max Length: 37

Field  39: Response Code
          Type: NUMERIC, Max Length: 2

Field  55: ICC System Related Data
          Type: LLLVAR, Max Length: 999



## Fixed vs Variable Length Fields

### Fixed Length Fields

Fixed length fields always have the same size. The parser reads exactly that many characters.

In [5]:
# Field 3 (Processing Code) is fixed length: 6 digits
field3_def = get_field_definition(3)
print(f"Field 3: {field3_def.description}")
print(f"Type: {field3_def.field_type.name}")
print(f"Length: {field3_def.max_length} (fixed)")
print("\nExample values:")
print("  000000 - Purchase")
print("  010000 - Cash withdrawal")
print("  200000 - Refund")

Field 3: Processing Code
Type: NUMERIC
Length: 6 (fixed)

Example values:
  000000 - Purchase
  010000 - Cash withdrawal
  200000 - Refund


### Variable Length Fields (LLVAR, LLLVAR)

Variable length fields have a length prefix:
- **LLVAR**: 2-digit length prefix (max 99 chars)
- **LLLVAR**: 3-digit length prefix (max 999 chars)

In [6]:
# Field 2 (PAN) is LLVAR
field2_def = get_field_definition(2)
print(f"Field 2: {field2_def.description}")
print(f"Type: {field2_def.field_type.name}")
print(f"Max Length: {field2_def.max_length}")
print("\nIn raw message: '164111111111111111'")
print("  '16' = length prefix (16 digits)")
print("  '4111111111111111' = actual PAN")

Field 2: Primary Account Number (PAN)
Type: LLVAR
Max Length: 19

In raw message: '164111111111111111'
  '16' = length prefix (16 digits)
  '4111111111111111' = actual PAN


## Parsing Examples

### Basic Authorization Request

In [7]:
# Build a sample message first
builder = ISO8583Builder()
parser = ISO8583Parser()

auth_request = ISO8583Message(
    mti="0100",
    fields={
        0: "0100",
        2: "4111111111111111",
        3: "000000",
        4: "000000005000",
        11: "123456",
        14: "2512",  # Expiry date YYMM
        22: "051",  # POS entry mode
        23: "001",  # Card sequence number
        41: "TERM0001",
        42: "MERCHANT123456 ",
        49: "840",  # Currency code (USD)
    },
)

raw = builder.build(auth_request)
print(f"Raw message: {raw}")
print(f"Length: {len(raw)} characters")

Raw message: 01007024060000C080001641111111111111110000000000000050001234562512051001TERM0001MERCHANT123456 840
Length: 98 characters


In [8]:
# Parse the message
parsed = parser.parse(raw)

print(f"MTI: {parsed.mti}")
print(f"Bitmap: {parsed.bitmap}")
print("\nParsed Fields:")
print("-" * 50)
for field_num in sorted(parsed.fields.keys()):
    if field_num > 0:
        field_def = get_field_definition(field_num)
        value = parsed.fields[field_num]
        print(f"F{field_num:03d} [{field_def.field_type.name:5s}] {field_def.description[:30]:30s} = {value}")

MTI: 0100
Bitmap: 7024060000C08000

Parsed Fields:
--------------------------------------------------
F002 [LLVAR] Primary Account Number (PAN)   = 4111111111111111
F003 [NUMERIC] Processing Code                = 000000
F004 [NUMERIC] Amount, Transaction            = 000000005000
F011 [NUMERIC] Systems Trace Audit Number (ST = 123456
F014 [NUMERIC] Date, Expiration (YYMM)        = 2512
F022 [NUMERIC] Point of Service Entry Mode    = 051
F023 [NUMERIC] Card Sequence Number           = 001
F041 [ALPHANUMERIC] Card Acceptor Terminal ID      = TERM0001
F042 [ALPHANUMERIC] Card Acceptor ID Code          = MERCHANT123456 
F049 [NUMERIC] Currency Code, Transaction     = 840


### Parsing with Network-Specific Fields

Different networks (Visa, Mastercard) may have different field definitions:

In [9]:
# Parse with Visa-specific field definitions
visa_parser = ISO8583Parser()
visa_parsed = visa_parser.parse(raw, network=CardNetwork.VISA)

print("Parsed with VISA network context:")
print(f"MTI: {visa_parsed.mti}")
print(f"Network: {visa_parsed.network}")

Parsed with VISA network context:
MTI: 0100
Network: CardNetwork.VISA


## Error Handling

The parser raises exceptions for invalid messages:

In [10]:
from iso8583sim.core.types import ISO8583Error

# Try parsing an invalid message
invalid_messages = [
    "0100",  # Too short - no bitmap
    "XXXX0000000000000000",  # Invalid MTI
    "0100ZZZZZZZZZZZZZZZZ",  # Invalid bitmap (non-hex)
]

for invalid in invalid_messages:
    try:
        parser.parse(invalid)
    except (ISO8583Error, ValueError) as e:
        print(f"Message: {invalid[:20]}...")
        print(f"Error: {e}")
        print()

Failed to parse message: Message too short for bitmap


Failed to parse message: Invalid MTI format - must be numeric


Failed to parse message: invalid literal for int() with base 16: 'ZZZZZZZZZZZZZZZZ'


Message: 0100...
Error: Failed to parse message: Message too short for bitmap

Message: XXXX0000000000000000...
Error: Failed to parse message: Invalid MTI format - must be numeric

Message: 0100ZZZZZZZZZZZZZZZZ...
Error: Failed to parse message: invalid literal for int() with base 16: 'ZZZZZZZZZZZZZZZZ'



## Performance Tips

For high-throughput parsing:

In [11]:
import time

# Reuse parser instance (caches field definitions)
parser = ISO8583Parser()

# Generate test messages
test_messages = []
for i in range(1000):
    msg = ISO8583Message(
        mti="0100",
        fields={
            0: "0100",
            2: f"411111111111{i:04d}",
            3: "000000",
            4: f"{i:012d}",
            11: f"{i:06d}",
        },
    )
    test_messages.append(builder.build(msg))

# Benchmark parsing
start = time.perf_counter()
for raw_msg in test_messages:
    parsed = parser.parse(raw_msg)
elapsed = time.perf_counter() - start

print(f"Parsed {len(test_messages)} messages in {elapsed:.3f}s")
print(f"Throughput: {len(test_messages)/elapsed:,.0f} messages/second")

Parsed 1000 messages in 0.005s
Throughput: 211,573 messages/second


## Next Steps

- **[03_building_messages.ipynb](03_building_messages.ipynb)** - Learn to build messages from scratch
- **[04_network_specifics.ipynb](04_network_specifics.ipynb)** - Network-specific parsing rules
- **[05_emv_data.ipynb](05_emv_data.ipynb)** - Parsing EMV/chip card data in Field 55